In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv('events.csv',low_memory=False)

## Cantidad de veces que el evento fue realizado por primera vez por una persona

In [20]:
personEv=df.loc[:,['person','event']]
primerEv=personEv.groupby('person').first()
#primerEv.loc[primerEv['event']=='conversion']
primerEv['event'].value_counts()

visited site         11519
ad campaign hit       5598
search engine hit     4648
generic listing       2575
viewed product        1866
checkout               564
brand listing          485
conversion             185
lead                   110
searched products       47
staticpage              27
Name: event, dtype: int64

## Cuantas personas realizaron una compra

In [38]:
primerEv['columnaNueva']=1
primerEv=personEv.groupby(['person','event']).count()
#primerEv.loc[primerEv['event']=='conversion']
primerEv=primerEv.reset_index()
cantidadConversiones=primerEv.loc[primerEv['event']=='conversion',:]
cantidadConversiones['event'].value_counts()

checkout             27624
visited site         26899
viewed product       26316
ad campaign hit      21312
search engine hit    17205
generic listing      15404
brand listing        10821
searched products     7420
staticpage            1475
conversion             716
lead                   291
Name: event, dtype: int64